# User Guide

The Euromod Conector is a Python library providing tools for running simulations and interacting with the tax-benefit microsimulation model [EUROMOD](https://euromod-web.jrc.ec.europa.eu "https://euromod-web.jrc.ec.europa.eu"). 

## Installation
The Euromod Connector can be installed from [PyPi](https://test.pypi.org/project/euromod/)  using _pip_:
```bash
$ pip install euromod
```

### Requirements
In order to run the model, we require two components: 1) the model (coded policy rules), and 2) the input microdata with the variables that respect the EUROMOD naming conventions.
For more information, please, read the sections "Model" and "Input microdata" on the [Download Euromod](https://euromod-web.jrc.ec.europa.eu/download-euromod "https://euromod-web.jrc.ec.europa.eu/download-euromod") web page.

## Running and navigating the model
The euromod package is object oriented and evolves around using the `Model` class that loads a representation of the EUROMOD model. Creating a euromod object works as follows

In [1]:
from euromod import Model

Create an object of the `core.Model` class by passing  the path to the  EUROMOD project: 

In [2]:
mod=Model(r"C:\EUROMOD_RELEASES_I6.0+")
mod

------------------------------
Model
------------------------------
	 countries: 28 elements
	 extensions: 11 elements
	 model_path: 'C:\\EUROMOD_RELEASES_I6.0+'

Note that every object that is related to the EUROMOD project comes with an informative description. Here we can see that the model has 3 relevant attributes to the user:
- countries
- extensions
- model_path
  
The countries and extensions attributes say that they contain elements. If we take a look at countries

mod.countriesies

The `Model.countries` attirbute is a container storing the `core.Country` objects which nest the country-specific tax-benefit policies and systems.

Displays the EUROMOD default countries:

In [3]:
mod.countries

0: AT 
1: BE 
2: BG 
3: CY 
4: CZ 
5: DE 
6: DK 
7: EE 
8: EL 
9: ES 
10: FI
11: FR
12: HR
13: HU
14: IE
15: IT
16: LT
17: LU
18: LV
19: MT
20: NL
21: PL
22: PT
23: RO
24: SE
25: SI
26: SK
27: SL

Displaying a specific country model, e.g. Simpleland, using the two-letters country code index:

We see indeed that the euromod model contains 28 countries. In a similar fashion we could have looked what kind of extensions that are stored in the model. The countries container can be indexed by both the number of the element and the country shortcode. Let us take a look at Sweden.

In [4]:
mod.countries["SE"]

------------------------------
Country
------------------------------
	 datasets: 27 elements
	 extensions: 12 elements
	 local_extensions: COVID
	 name: 'SE'
	 policies: 26 elements
	 systems: 18 elements

Here we see again an informative representation of the `Country` object, which contains several attributes that can be accessed. In order to simulate a system we run a specific `System`. We can obtain the system `Container` of the `Country` Sweden

In [5]:
mod.countries["SE"].systems

0: SE_2006 
1: SE_2007 
2: SE_2008 
3: SE_2009 
4: SE_2010 
5: SE_2011 
6: SE_2012 
7: SE_2013 
8: SE_2014 
9: SE_2015 
10: SE_2016
11: SE_2017
12: SE_2018
13: SE_2019
14: SE_2020
15: SE_2021
16: SE_2022
17: SE_2023

### Running a simulation

In order to run the tax system we need a dataset that fits the requirement to use.In case you already know which dataset you want to use one can simply load the data and run the model as follows:.

In [6]:
import pandas as pd
data=pd.read_csv(r"C:\EUROMOD_RELEASES_I6.0+\Input\SE_2021_b1.txt",sep="\t")
out_baseline = mod.countries["SE"].systems["SE_2021"].run(data,"SE_2021_b1")
out_baseline

Simulation for system SE_2021 with dataset SE_2021_b1 finished.


------------------------------
Simulation
------------------------------
	 constantsToOverwrite: {}
	 errors: []
	 output_filenames: ['se_2021_std.txt']
	 outputs: Pandas DataFrame of 240 variables and 21671 observations.

In [7]:
pd.set_option('display.max_rows', 11, 'display.max_columns',7)

Note that the run function here takes the mandatory argument `dataset_id`, which in our case is here SE_2022_b1. This is necessary such that EUROMOD can apply the dataset specific logic with respect to setting default values and uprating. This returned us a Simulation object with multiple attributes returned. The one of interest here is outputs, which contains the outputdataset(s) returned by the microsimulation model.

In [8]:
out_baseline.outputs[0]

,idhh,idperson,idmother,...,tu_bho_se_IsDependentChild,tu_bho_se_IsLoneParent,tu_bho_se_IsPartner
0,200.0,20001.0,0.0,...,0.0,0.0,0.0
1,300.0,30001.0,0.0,...,0.0,0.0,0.0
2,300.0,30002.0,0.0,...,0.0,0.0,1.0
3,500.0,50001.0,0.0,...,0.0,0.0,0.0
4,500.0,50002.0,0.0,...,0.0,0.0,1.0
...,...,...,...,...,...,...,...
21666,1936500.0,193650002.0,0.0,...,0.0,0.0,1.0
21667,1936800.0,193680001.0,0.0,...,0.0,0.0,1.0
21668,1936800.0,193680002.0,0.0,...,0.0,0.0,0.0
21669,1936800.0,193680003.0,193680002.0,...,1.0,0.0,0.0


### Navigating the model
The `Model` object actually contains a full representation of the model that can be accessed using it's attributes.  The implementation in Python mimicks the hierarchical structure of the EUROMOD User Interface. A full description of the available types can be found in the API reference.
#### The spine
The spine of EUROMOD is what represents the series of calculations with respect taxes and benefits.
The spine consists out of three hierarchically ordered elements:
- Policy
    - Function
        - Parameter
     
The connector mimmicks this hierarchical implementation through an object-oriented representation. The three hierarchical elements are defined on the `Country` level and implemented on the `System` level.

Let us take a look at the policies, which are an attribute of the `Country` object.

In [9]:
mod.countries["SE"].policies

0: setdefault_se         |                                   |    DEF: SET DEFAULT 
1: uprate_se             |                                   |    DEF: UPRATING FACTORS 
2: ConstDef_se           |                                   |    DEF: CONSTANTS 
3: IlsDef_se             |                                   |    DEF: INCOME CONCEPTS (standardized) 
4: IlsUDBdef_se          |                                   |    DEF: INCOME CONCEPTS (UDB) 
5: ildef_se              |                                   |    DEF: INCOME CONCEPTS (non-standardized) 
6: random_se             |                                   |    DEF: Random assignment 
7: TransLMA_se           |                                   |    DEF: Modelling labour market transitions (DO NOT S ... 
8: tudef_se              |                                   |    DEF: ASSESSMENT UNITS 
9: yem_se                |  (with switch set for MWA)        |    DEF: minimum wage 
10: neg_se               |                             

As one can see the `policies` attribute is a Container object here, and it's elements which are of the type `Policy` here are accessible.

In [10]:
mod.countries["SE"].policies[22]

------------------------------
Policy
------------------------------
	 ID: '22DDB39D-1393-4196-AC79-23825D0D6DF2'
	 comment: 'BEN: Housing allowance for pensioners'
	 extensions: 0 elements
	 functions: 19 elements
	 name: 'bhope_se'
	 order: '23'
	 private: 'no'
	 spineOrder: '23'

The implementation of a policy is accessible through the `System` object.

In [11]:
mod.countries["SE"].systems["SE_2021"].policies

0: setdefault_se         | on                                   |    DEF: SET DEFAULT 
1: uprate_se             | on                                   |    DEF: UPRATING FACTORS 
2: ConstDef_se           | on                                   |    DEF: CONSTANTS 
3: IlsDef_se             | on                                   |    DEF: INCOME CONCEPTS (standardized) 
4: IlsUDBdef_se          | on                                   |    DEF: INCOME CONCEPTS (UDB) 
5: ildef_se              | on                                   |    DEF: INCOME CONCEPTS (non-standardized) 
6: random_se             | on                                   |    DEF: Random assignment 
7: TransLMA_se           | off                                  |    DEF: Modelling labour market transitions (DO NOT S ... 
8: tudef_se              | on                                   |    DEF: ASSESSMENT UNITS 
9: yem_se                | off (with switch set for MWA)        |    DEF: minimum wage 
10: neg_se               

Here we see that some policies are turned off by default. Note that the behaviour of the policies can be controlled from the connector. We can for example switch the policy bunct_se to on. Let us first look at the policy

In [12]:
mod.countries["SE"].systems["SE_2021"].policies[22]

------------------------------
PolicyInSystem
------------------------------
	 ID: 'bde78132-3f44-4d2e-a1ea-4849f88c277622DDB39D-1393-4196-AC79-23825D0D6DF2'
	 comment: 'BEN: Housing allowance for pensioners'
	 extensions: 0 elements
	 functions: 19 elements
	 name: 'bhope_se'
	 order: '23'
	 polID: '22DDB39D-1393-4196-AC79-23825D0D6DF2'
	 private: 'no'
	 spineOrder: '23'
	 switch: 'on'
	 sysID: 'bde78132-3f44-4d2e-a1ea-4849f88c2776'

We see here that there is this attribute `switch` that is part of the `PolicyInSystem` class. This attribute, similar to attribute is modifiable and the changes that you will make will be passed to the EUROMOD software when simulating. Changes implemented during your Python session are however not lasting. For now, when making lasting changes to the model, one should implement them via the User Interface of EUROMOD. Note that the python connector is not checking what kind of modifications you make to the model. Changing attributes like ID's are definitely not adviseable. 

In [13]:
mod.countries["SE"].systems["SE_2021"].policies[13].switch = 'on'
out_with_bunct_se = mod.countries["SE"].systems["SE_2021"].run(data,"SE_2021_b1")
# Here we see that the average benefit, which is represented by ils_ben,  is indeed larger by switching on the bunct_se policy.
out_baseline.outputs[0].ils_ben.mean() - out_with_bunct_se.outputs[0].ils_ben.mean()

Simulation for system SE_2021 with dataset SE_2021_b1 finished.


0.0

In [14]:
mod.countries["SE"].systems["SE_2021"].policies

0: setdefault_se         | on                                   |    DEF: SET DEFAULT 
1: uprate_se             | on                                   |    DEF: UPRATING FACTORS 
2: ConstDef_se           | on                                   |    DEF: CONSTANTS 
3: IlsDef_se             | on                                   |    DEF: INCOME CONCEPTS (standardized) 
4: IlsUDBdef_se          | on                                   |    DEF: INCOME CONCEPTS (UDB) 
5: ildef_se              | on                                   |    DEF: INCOME CONCEPTS (non-standardized) 
6: random_se             | on                                   |    DEF: Random assignment 
7: TransLMA_se           | off                                  |    DEF: Modelling labour market transitions (DO NOT S ... 
8: tudef_se              | on                                   |    DEF: ASSESSMENT UNITS 
9: yem_se                | off (with switch set for MWA)        |    DEF: minimum wage 
10: neg_se               

As mentioned earlier, the connector mimicks the hierarchical structure of the UI. Hence, the definition of functions and parameters are defined on the country level, and their actual implementation are here also accessible via the Tax System. Note that also here, the values of a parameter and the switch of a function can be manipulated through the Python Connector without saving the changes permanently.

In [15]:
# Overview of the functions defined in the bho_se policy
print(mod.countries["SE"].policies[22].functions)
# System specific Implementation of functions
print(mod.countries["SE"].systems["SE_2021"].policies[22].functions)

0: DefVar       |    Temporary variables for Housing Allowance for pens ... 
1: Elig         |    Living with partner 
2: ArithOp      |    Wealth to be included in the means 
3: Elig         |    Living without partner 
4: ArithOp      |    Wealth to be included in the means 
5: Allocate     |    Allocation of wealth to the partners 
6: Elig         |    Elderly or disabled adult (i.e. head or partner) 
7: BenCalc      |    "Reserved amount (""income disregard"")" 
8: BenCalc      |    Change in definition of Income Means for for Housi ... 
9: DefIl        |    Income Means for Housing Allowance for pensioners 
10: BenCalc     |    Income of children is not take into account in the ... 
11: BenCalc     |    Maximum housing allowance 
12: BenCalc     |     
13: BenCalc     |    Maximum housing allowance 
14: BenCalc     |    Maximum housing allowance 
15: Allocate    |    Allocation of income to the partners 
16: BenCalc     |    Final housing allowance for pensioners 
17: Allocate    

In [16]:
# Overview of the parameters defined in the bho_se policy
print(mod.countries["SE"].policies[22].functions[0].parameters)
# Implementation of parameters
print(mod.countries["SE"].systems["SE_2021"].policies[22].functions[0].parameters)

0: i_means_bhope_prel

0: i_means_bhope_prel     | 0  



#### Other Important EUROMOD Objects
Central to the EUROMOD project, next to the coding of the policies is the microdata. How datasets should be treated by the model is configured in the model already. The attributes of the datasets are just like the spine-elements accessible and modifiable.

In [17]:
mod.countries["SE"].datasets

0: SE_2007_a4   
1: SE_2008_a3   
2: training_data
3: SE_2010_a1   
4: SE_2012_a2   
5: SE_2015_a1   
6: SE_2009_hhot 
7: SE_2010_hhot 
8: SE_2011_hhot 
9: SE_2012_hhot 
10: SE_2013_hhot
11: SE_2014_hhot
12: SE_2015_hhot
13: SE_2016_hhot
14: SE_2017_hhot
15: SE_2016_a1  
16: SE_2018_hhot
17: SE_2019_hhot
18: SE_2018_a2  
19: SE_2017_a3  
20: SE_2020_hhot
21: SE_2019_a1  
22: SE_2020_b1  
23: SE_2021_hhot
24: SE_2022_hhot
25: SE_2021_b1  
26: SE_2023_hhot

In the previous section we used SE_2022_b1. Let us have a look at it.

In [18]:
mod.countries["SE"].datasets["SE_2021_b1"]

------------------------------
Dataset
------------------------------
	 ID: 'c7b651ed-b311-4e39-80b4-18ca19957ce7'
	 coicopVersion: ''
	 comment: ''
	 currency: 'national'
	 decimalSign: '.'
	 name: 'SE_2021_b1'
	 private: 'no'
	 readXVariables: 'no'
	 useCommonDefault: 'no'
	 yearCollection: '2021'
	 yearInc: '2020'

Similar to policies, functions and parameters, can the attributes of the policies be modified here. An other important concept in euromod are extensions and are defined globally on the `Model` level or locally on the `Country` level. 

In [19]:
mod.countries["SE"].extensions[7]

------------------------------
Extension
------------------------------
	 ID: '557c232a-9ce6-4808-b52f-ca5e02fe8cf4'
	 look: '|BUTTON_COLOR=-16744384|'
	 name: 'Minimum Wage Adjustments'
	 shortName: 'MWA'

The `extensions` object is of the type `Container`, which means that you can index it by the number preceding their name. If we want to access the information stored in the Minimum Wage Adjustments extension for example, one can simply use the following command. 

Displaying information about a dataset relative to a system, e.g. dataset 'sl_demo_v4' for the system 'SL_1996', by indexing the attribute `datasets`:

In [20]:
mod.countries['SL'].systems['SL_1996'].datasets[0]

------------------------------
DatasetInSystem
------------------------------
	 ID: 'F7E5CACE-CECC-4BB6-9841-A936D0975481CBA7E428-F8E4-4CEB-8A5E-9ACE73987DD7'
	 bestMatch: 'yes'
	 coicopVersion: ''
	 comment: ''
	 currency: 'euro'
	 dataID: 'CBA7E428-F8E4-4CEB-8A5E-9ACE73987DD7'
	 decimalSign: '.'
	 name: 'sl_demo_v4'
	 private: 'no'
	 readXVariables: 'no'
	 sysID: 'F7E5CACE-CECC-4BB6-9841-A936D0975481'
	 useCommonDefault: 'no'